## Auteur : Mauchaussee Pauline 
# Date de création : 2023/08/17  
# Présentation :** Ce notebook permet de telecharger des données climatique depuis une liste de stations stockée dans un table BigQuerry et d'envoyer les données en format CSV dans un bucket GCP.

## Prérequis :
# - Sauvegarder le fichier excel sous forme de CSV
# - Un bucket gcp pour le stockage de données. (BUCKET_NAME)
# - Une table BigQuerry contenant la liste des station (TABLE_ID)
# - Un token API infoclimat


## Params:
# BUCKET_NAME : nom du bucket GCP, DOIT être dans le même projet.
# TABLE_IDE : ID de la table BigQuerry, DOIT être dans le même projet.
# KEY_PATH : Chemin vers le fichier JSON permettant de ce connecter au service account

## Import des librairies

In [28]:
from google.cloud import storage

In [29]:
import os

In [30]:
#Importer les bibliothèques nécessaires (pandas et pyspark) dans le script pyspark
import pandas as pd

In [31]:
# !pip install openpyxl

In [32]:
# !pip install xlrd

In [33]:
import openpyxl

In [34]:
from pyspark.sql.functions import lit, col, StringType

In [35]:
import pyspark.sql
from pyspark.sql import *

In [36]:
import requests

In [37]:
from pyspark import SparkFiles

In [38]:
import zipfile
import io
import os

# Création d'une fonction pour récupérer de la donnée à partir d'une URL 

In [39]:
def TelechargementData():

    url = "https://services.eaufrance.fr/documents/openData/SISPEA_FR_"+str(annee)+"_AEP.zip"

    reponse = requests.get(url)

    content = reponse.content

    # Création d'un objet ZipFile à partir du contenu ZIP téléchargé
    zip_file = zipfile.ZipFile(io.BytesIO(content))

    # Création du répertoire de destination s'il n'existe pas
    os.makedirs("Data", exist_ok=True)

    # Extraction des fichiers du fichier ZIP et enregistrement dans le répertoire de destination
    zip_file.extractall("Data")
    
    # Fermeture du fichier ZIP
    zip_file.close()

# Création d'une fonction pour lire mon fichier excel et conversion en un DataFrame Pandas

In [40]:
def lecturePandas(annee):
    ''' Cette fonction permet de lire un fichier excel et de le convertir en dataFrame Pandas'''

    #Pour lire le fichier excel on utilise Pandas 
    #On donne le chemin du fichier excel
    excel_file_path = "./Data/SISPEA_FR_"+str(annee)+"_AEP.xls"
    #On donne le nom de l'onglet
    sheet_name = "Entités de gestion"
    #Lecture du fichier excel avec pandas 
    pandas_df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

    return pandas_df

# Je sélectionne les colonnes que l'on souhaite garder

In [41]:
def selectioncolonne(pandas_df):
    '''Cette fonction selectionne les colonnes que l'on veut garder '''
    pandas_df_new = pandas_df[["N° SIREN","VP.224","VP.225", "VP.226", "VP.227","VP.228","VP.229","VP.231", "VP.232","VP.234"]]

    return pandas_df_new

# Création d'une fonction pour convertir la table pandas en table spark

In [42]:
def convertirPandasEnSpark(pandas_df):
    '''Cette fonction converti la table pandas en table spark'''
    #Conversion du DataFrame Pandas en DataFrame Spark
    table = spark.createDataFrame(pandas_df)
    
    return table

# Je renomme les colonnes de mon tableau

In [43]:
def renommageColonnes(table_spark):
    ''' Cette fonction renomme les colonnes de la table '''

    table_renamed = table_spark.withColumnRenamed("VP.224", "Indice linéaire de consommation")\
        .withColumnRenamed("VP.225", "Rendement sur les 3 années précédentes")\
        .withColumnRenamed("VP.226", "Rendement seuil par défaut")\
        .withColumnRenamed("VP.227", "Rendement seuil en ZRE")\
        .withColumnRenamed("VP.228", "Densité linéaire d'abonnés")\
        .withColumnRenamed("VP.229", "Ratio habitants par abonnés")\
        .withColumnRenamed("VP.231", "Consommation moyenne par abonné")\
        .withColumnRenamed("VP.232", "Volumes consommés comptabilisés")\
        .withColumnRenamed("VP.234", "Volume produit + Volume importé")
    return table_renamed

In [44]:
def typage(La_table_spark_que_je_veux_modifier):
    "cette fonction change le typage de la colonne numero Siren"
    La_nouvelle_table_spark = La_table_spark_que_je_veux_modifier.withColumn("N° SIREN", col("N° SIREN").cast(StringType()))
    return(La_nouvelle_table_spark)

# Ajout de la colonne année

In [45]:
def ajoutcol_annee(table_spark,annee):
    # Add new constanst column
    dataframe_colannee = table_spark.withColumn("Annee", lit(annee))
    print(type(dataframe_colannee))
    printSchema(dataframe_colannee)
    return dataframe_colannee

# Je sauvegarde mon tableau dans un fichier csv

In [46]:
def transformationCSV(table_spark):
    ''' Cette fonction enregistre une table spark au format CSV '''
    table_spark.write.csv("./Local Disk/Data/consommation_eau.csv", header=True, mode="append")

In [47]:
def transformationCSV_dans_bucket(table_spark,annee):
    ''' Cette fonction enregistre une table spark en csv dans un bucket GCP'''

    nom_fichier = f"consommation_eau.csv"
    chemin_gcs= f"gs://{NOM_BUCKET}/{NOM_DOSSIER_DANS_BUCKET}/{nom_fichier}"

    client = storage.Client()
    table_spark.write.format("csv").save(chemin_gcs, mode='append')

    print(f"Le fichier {nom_fichier} a été enregistré dans {chemin_gcs}")

# Fonction principale pour lancer toutes mes fonctions

In [48]:
def main(annee):
    
    TelechargementData()
    
    pandas_dataframe = lecturePandas(annee)

    pandas_dataframe_propre = selectioncolonne(pandas_dataframe)

    spark_dataframe = convertirPandasEnSpark(pandas_dataframe_propre)

    df_bons_noms = renommageColonnes(spark_dataframe)
    
    df_bons_types = typage(df_bons_noms)
    
    dataframe_colannee = ajoutcol_annee(df_bons_types, annee)

    transformationCSV_dans_bucket(dataframe_colannee, annee)
    

# Selection des années à traiter.

In [49]:
annee_min = 2008
anne_max = 2021

In [50]:
# Nom du bucket de stockage
NOM_BUCKET = "code_de_source_lake"
# Nom du dossier dans le bucket GCS
NOM_DOSSIER_DANS_BUCKET = 'Output_data_consommationeau'


# Execution du programme

In [51]:
for annee in range(annee_min,anne_max+1):
    main(annee)

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 44024)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/miniconda3/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/miniconda3/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/miniconda3/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/usr/lib/spark/python/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/usr/lib/spark/python/pyspark/accumulators.py", line 253, in poll
    if func():
  File "/usr/lib/spark/python/pyspark/accumulators.py", line 257, in accum_updates
    num_updates = read_int(self.r

Py4JError: SparkSession does not exist in the JVM